In [28]:
import numpy as np

In [80]:
#Environment for testing
def initialize_reward(states,goal):
    reward = {}
    for s in states:
        if s == goal:
            reward[s] = 1
        else:
            reward[s] = -1
    return reward


def testing_environment():
    actionspace = {'up' : (0,1), 'down' : (0,-1), 'left' : (-1,0), 'right' : (1,0),'none' :(0,0)}

   

    def transition (state,action):
        next_state = tuple(np.asarray(state) + np.asarray(actionspace[action]))
        if (state,next_state) in barriers or (next_state,state) in barriers:
            return state
        return next_state

    def actions(new_state):
        actions = {}
        if new_state[0] > 1:
            actions['left'] = (-1,0)
        if new_state[0]  < maxcol:
            actions['right'] = (1,0)
        if new_state[1] > 1:
            actions['down'] = (0,-1)
        if new_state[1]  < maxrow:
            actions['up'] = (0,1)
        actions['none'] = (0,0)
        return actions

    def transition_prob(s_next,s,a):
        return s_next == s
    return transition, actions, transition_prob

In [77]:
#Value Iteration for the policies
def Compute_Value_Function(states,actions,reward):
    value_func = {}
    for s in states:
        value_func[s] = 0
    error = 1
    while error > 0:
        error = 0
        for s in states:
            value = value_func[s]
            value_func[s] = Return_Maximum_Action(s,value_func,actions(s),reward)[0]
            error = max(error, abs(value - value_func[s]))
    return value_func


def Return_Maximum_Action(s,value_func,actions,reward):
    action  = 0
    new_value = -1000
    for a in actions.keys():    
        expected =  (gamma*value_func[transition(s,a)]+reward[s]) + (1-p)*(sum([(value_func[transition(s,b)]+reward[transition(s,b)]) for b in actions.keys() if b != a]))
        if expected > new_value:
            new_value = expected
            action = a

    
    return (new_value, action)

def optimal_action(s,value_func,actions,beta,reward):
    out = []
    total = 0
    for a in actions.keys():    
        q =  (gamma*p*value_func[transition(s,a)]+reward[s]) + (1-p)*(sum([(value_func[transition(s,b)]+reward[transition(s,b)]) for b in actions.keys() if b != a]))
        out.append((a,np.exp(beta*q)))
        total += np.exp(beta*q)
    return [(x,y/total) for x,y in out]
        
def Compute_Policy(value_func,reward):
    policy_dict = {}
    for s in states:
        policy_dict[s] = optimal_action(s,value_func,actions(s),b,reward)
    return policy_dict

In [49]:
def goal_inference(sequence, goals, goal_prob, policy,transition_prob):
    goal_probabilities = []
    for g in goals:
        p_seq = 1
        for s in sequence[1:]:
            p_snext = 0
            for a,action_prob in policy[g][s]:
                p_snext += transition_prob(transition(s,a),s,a)*action_prob
            p_seq *= p_snext
        goal_probabilities.append((g,p_seq))
    return goal_probabilities

In [90]:
#Main Cell: Executes the model
goals = [(2,10),(4,4),(5,5),(6,6)]
gamma = 0.9
p = 1
b = 1
maxrow = 19
maxcol = 7
barriers = []
sequence = [(1,2),(1,3),(2,3),(3,3),(4,3)]
output = []
states = [(x,y) for x in range(1,maxcol+1) for y in range(1,maxrow+1)]
transition, actions, transition_prob = testing_environment()
rewards = {}
value = {}
policy = {}
for g in goals:
    rewards[g] = initialize_reward(states,g)
    value[g] = Compute_Value_Function(states,actions,rewards[g])
    policy[g] = Compute_Policy(value[g],rewards[g])
output.append(goal_inference(sequence, goals, 1/len(goals), policy,transition_prob))
output

[[((2, 10), 0.0007866250271018762),
  ((4, 4), 0.00012772720748122307),
  ((5, 5), 0.00018539107232044965),
  ((6, 6), 0.00036932825832728804)]]